In [6]:
import IPython
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import librosa
import sys ; sys.path.append('../')
from model.network_architecture import autoencoder
from tqdm import tqdm
from evaluacion import *

In [7]:
PESOS = '/media/martin/martindrive/ckpts/reales/weights.02-0.009.hdf5'
modelo = autoencoder()
modelo.load_weights(PESOS)

Busco formar baches para hacer predicciones de a grupos y guardarlas en RAM

In [8]:
clean_list = get_audio_list('/home/martin/deep-dereverb/data/test/clean/')
real_list = get_audio_list('/home/martin/deep-dereverb/data/test/real/')
clean_list.sort(); real_list.sort()

In [9]:
BACHE = len(clean_list) // 10

entradas_clean = np.empty((BACHE, 256, 256))                                                                                                                                                                
entradas_reverb = np.empty((BACHE, 256, 256))
scale_spec = np.empty((BACHE, 4)) #clean, reverb

for i, (clean_path, reverb_path) in enumerate(zip(clean_list[:BACHE], real_list[:BACHE])):                                                                                          
    # Cargo los audios
    reverb, _ = librosa.load(reverb_path, sr=None)
    clean, _ = librosa.load(clean_path, sr=None) 
    # Obtengo los espectros
    reverb_spec, reverb_min, reverb_max = gen_stft(reverb)                                                                                
    clean_spec, clean_min, clean_max = gen_stft(reverb)                                                                                
    
    # Guardo los valores
    entradas_clean[i], entradas_reverb[i] = clean_spec, reverb_spec                                                                                     
    scale_spec[i] = clean_min, clean_max, reverb_min, reverb_max

    
#obtener los dereverb pasando por la red
dereverb_spec = modelo.predict(entradas_reverb, batch_size=8)

In [10]:
dereverb_spec.shape

(826, 256, 256, 1)

In [11]:
def metric_from_spec(task):
    clean_spec, reverb_spec, dereverb_spec, scale = task[0], task[1], task[2], task[3]
    fs=16000
    clean_spec = clean_spec.reshape(256,256)
    reverb_spec = reverb_spec.reshape(256,256)
    dereverb_spec = dereverb_spec.reshape(256,256)
    
    clean = frame_to_raw(clean_spec, scale[0], scale[1])
    reverb = frame_to_raw(reverb_spec, scale[2], scale[3])
    dereverb = frame_to_raw(dereverb_spec, scale[2], scale[3])
    
    SRMR = srmr.srmr(reverb, fs)[0]                                                                                                     
    SDR, _, _, _ = mir_eval.separation.bss_eval_sources(clean, reverb, compute_permutation=True)                                        
    ESTOI = pystoi.stoi(clean, reverb, fs, extended = True)                                                                             
    metrics_reverb = [SRMR, SDR[0], ESTOI]
    
    SRMR = srmr.srmr(dereverb, fs)[0]                                                                                                     
    SDR, _, _, _ = mir_eval.separation.bss_eval_sources(clean, dereverb, compute_permutation=True)                                        
    ESTOI = pystoi.stoi(clean, dereverb, fs, extended = True)                                                                             
    metrics_dereverb = [SRMR, SDR[0], ESTOI]
    return metrics_reverb, metrics_dereverb

In [12]:
tasks =  [[entradas_clean[i],entradas_reverb[i],dereverb_spec[i],scale_spec[i]] for i in range(BACHE)]

In [ ]:
import multiprocessing as mp
from tqdm import tqdm

pool = mp.Pool(processes=8)
resultados = []

res = [pool.apply_async(metric_from_spec, args=(tasks[i],)) for i in range(BACHE)]
res = [r.get() for r in tqdm(res)]
pool.close()
pool.join()

 14%|█▍        | 116/826 [12:25<1:22:36,  6.98s/it]Process ForkPoolWorker-3:
Process ForkPoolWorker-8:
Process ForkPoolWorker-1:
Process ForkPoolWorker-7:
Traceback (most recent call last):
Process ForkPoolWorker-4:
Traceback (most recent call last):
Process ForkPoolWorker-5:
Traceback (most recent call last):
  File "/home/martin/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/martin/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/martin/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/martin/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/martin/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target

KeyboardInterrupt
Process ForkPoolWorker-2:
Traceback (most recent call last):
  File "/home/martin/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/martin/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/martin/anaconda3/lib/python3.8/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "<ipython-input-11-a2e321a218da>", line 18, in metric_from_spec
    SDR, _, _, _ = mir_eval.separation.bss_eval_sources(clean, dereverb, compute_permutation=True)
Process ForkPoolWorker-6:
Traceback (most recent call last):
  File "/home/martin/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/martin/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/martin/anaconda3/lib/python3.8/site-packages